<a href="https://colab.research.google.com/github/EdnahM/NLP_POS_Luhya/blob/dev/POS_Luhya_Imbalanced_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Deep Learning Nueral Network solving imbalanced data by class weighting


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, TimeDistributed, Dense
from tensorflow.keras.initializers import Orthogonal, RandomUniform, GlorotUniform, Zeros
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

In [ ]:
import os
os.listdir()

['.config', 'sample_data']

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
data =  pd.read_csv("/content/drive/MyDrive/MSC-DS-2023/cleaned_data.csv", header=0)

In [ ]:
words = list(set(data["WORD"].values))
n_words = len(words)
tags = list(set(data["SPEECH TAG"].values))
n_tags = len(tags)

In [ ]:

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0

In [ ]:
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {i: t for t, i in tag2idx.items()}


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

# Load the dataset
# data = pd.read_csv('cleaned_data.csv')
data =  pd.read_csv("/content/drive/MyDrive/MSC-DS-2023/cleaned_data.csv", header=0)

# Prepare word and tag mappings
words = list(set(data["WORD"].values))
n_words = len(words)
tags = list(set(data["SPEECH TAG"].values))
n_tags = len(tags)

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0

tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {i: t for t, i in tag2idx.items()}

# Group words and tags into sentences
sentences = []
sentence_tags = []

temp_sentence = []
temp_tags = []

for index, row in data.iterrows():
    word = row['WORD']
    tag = row['SPEECH TAG']
    if word == ".":
        if temp_sentence:  # if the sentence is not empty
            sentences.append(temp_sentence)
            sentence_tags.append(temp_tags)
            temp_sentence = []
            temp_tags = []
    else:
        temp_sentence.append(word)
        temp_tags.append(tag)

# Add the last sentence if it's not empty
if temp_sentence:
    sentences.append(temp_sentence)
    sentence_tags.append(temp_tags)

# Convert words and tags to indices
X = [[word2idx.get(w, word2idx["UNK"]) for w in s] for s in sentences]
y = [[tag2idx[t] for t in s] for s in sentence_tags]

# Pad sequences
maxlen = 50  # Choose a suitable maximum length for your sequences
X = pad_sequences(X, maxlen=maxlen, padding='post')
y = pad_sequences(y, maxlen=maxlen, padding='post')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Calculate class weights
y_flat = [item for sublist in y_train for item in sublist]  # Flatten the list of lists
class_weights = compute_class_weight('balanced', classes=np.unique(y_flat), y=y_flat)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Convert y_train and y_test to categorical
y_train = np.array([to_categorical(i, num_classes=n_tags) for i in y_train])
y_test = np.array([to_categorical(i, num_classes=n_tags) for i in y_test])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, TimeDistributed, Dense

# Define model architecture
model = Sequential()
model.add(Embedding(input_dim=n_words + 2, output_dim=50, input_length=maxlen, mask_zero=True))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(TimeDistributed(Dense(n_tags, activation="softmax")))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, class_weight=class_weights_dict)

# Save the model
model.save('pos_model.h5')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, TimeDistributed, Dense

# Define model architecture
model = Sequential()
model.add(Embedding(input_dim=n_words + 2, output_dim=50, input_length=maxlen, mask_zero=True))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(TimeDistributed(Dense(n_tags, activation="softmax")))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, class_weight=class_weights_dict)

# Save the model
model.save('pos_model.h5')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, TimeDistributed, Dense, Dropout

# Define model architecture
model = Sequential()
model.add(Embedding(input_dim=n_words + 2, output_dim=50, input_length=maxlen, mask_zero=True))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(TimeDistributed(Dense(n_tags, activation="softmax")))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with class weights
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, class_weight=class_weights_dict)

# Save the model
model.save('pos_model_improved.h5')


Epoch 1/100
422/422 [==============================] - 60s 117ms/step - loss: 4.5700 - accuracy: 0.3272 - val_loss: 1.2212 - val_accuracy: 0.6656
Epoch 2/100
422/422 [==============================] - 46s 110ms/step - loss: 0.8286 - accuracy: 0.7705 - val_loss: 0.4801 - val_accuracy: 0.8277
Epoch 3/100
422/422 [==============================] - 47s 111ms/step - loss: 0.2700 - accuracy: 0.8970 - val_loss: 0.2591 - val_accuracy: 0.9297
Epoch 4/100
422/422 [==============================] - 46s 110ms/step - loss: 0.1288 - accuracy: 0.9542 - val_loss: 0.1527 - val_accuracy: 0.9626
Epoch 5/100
422/422 [==============================] - 47s 111ms/step - loss: 0.0665 - accuracy: 0.9729 - val_loss: 0.1083 - val_accuracy: 0.9752
Epoch 6/100
422/422 [==============================] - 48s 113ms/step - loss: 0.0464 - accuracy: 0.9799 - val_loss: 0.0910 - val_accuracy: 0.9786
Epoch 7/100
422/422 [==============================] - 48s 113ms/step - loss: 0.0262 - accuracy: 0.9862 - val_loss: 0.0676 -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/100
422/422 [==============================] - 117s 257ms/step - loss: 4.3985 - accuracy: 0.3642 - val_loss: 1.1624 - val_accuracy: 0.6895
Epoch 2/100
422/422 [==============================] - 109s 259ms/step - loss: 0.8113 - accuracy: 0.7765 - val_loss: 0.4554 - val_accuracy: 0.8355
Epoch 3/100
422/422 [==============================] - 108s 255ms/step - loss: 0.3369 - accuracy: 0.8919 - val_loss: 0.2612 - val_accuracy: 0.9352
Epoch 4/100
422/422 [==============================] - 107s 254ms/step - loss: 0.1763 - accuracy: 0.9455 - val_loss: 0.1919 - val_accuracy: 0.9562
Epoch 5/100
422/422 [==============================] - 107s 254ms/step - loss: 0.1319 - accuracy: 0.9589 - val_loss: 0.1554 - val_accuracy: 0.9656
Epoch 6/100
422/422 [==============================] - 106s 252ms/step - loss: 0.0831 - accuracy: 0.9690 - val_loss: 0.1165 - val_accuracy: 0.9745
Epoch 7/100
422/422 [==============================] - 107s 254ms/step - loss: 0.0555 - accuracy: 0.9760 - val_loss: 0

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Load the model
model = load_model('pos_model_improved.h5')

def predict_with_confidence(sentence):
    words = sentence.split()
    word_indices = [word2idx.get(word, word2idx["UNK"]) for word in words]
    word_indices_padded = pad_sequences([word_indices], maxlen=maxlen, padding='post')
    predictions = model.predict(word_indices_padded)

    predicted_tags = []
    confidence_scores = []

    for pred in predictions[0]:
        tag_idx = np.argmax(pred)
        predicted_tags.append(idx2tag[tag_idx])
        confidence_scores.append(pred[tag_idx])

    return predicted_tags[:len(words)], confidence_scores[:len(words)]


In [ ]:
sentence = "Amosi we munaanie Bibilia"
predicted_tags, confidence_scores = predict_with_confidence(sentence)
print("Predicted Tags:", predicted_tags)
print("Confidence Scores:", confidence_scores)

1/1 [==============================] - 1s 866ms/step
Predicted Tags: ['N', 'CONJ', 'N', 'N']
Confidence Scores: [0.99997276, 0.94958043, 0.99993557, 0.99992865]


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define model architecture
model = Sequential()
model.add(Embedding(input_dim=n_words + 2, output_dim=50, input_length=maxlen, mask_zero=True))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(TimeDistributed(Dense(n_tags, activation="softmax")))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.1, class_weight=class_weights_dict, callbacks=[early_stopping])

# Save the model
model.save('pos_model_with_early_stopping.h5')



Epoch 1/50
422/422 [==============================] - 120s 263ms/step - loss: 4.5913 - accuracy: 0.3488 - val_loss: 1.3039 - val_accuracy: 0.6511
Epoch 2/50
422/422 [==============================] - 108s 257ms/step - loss: 1.0368 - accuracy: 0.7486 - val_loss: 0.5776 - val_accuracy: 0.8065
Epoch 3/50
422/422 [==============================] - 111s 264ms/step - loss: 0.4125 - accuracy: 0.8706 - val_loss: 0.3154 - val_accuracy: 0.9207
Epoch 4/50
422/422 [==============================] - 107s 253ms/step - loss: 0.2281 - accuracy: 0.9323 - val_loss: 0.2211 - val_accuracy: 0.9500
Epoch 5/50
422/422 [==============================] - 106s 252ms/step - loss: 0.1335 - accuracy: 0.9565 - val_loss: 0.1546 - val_accuracy: 0.9666
Epoch 6/50
422/422 [==============================] - 107s 253ms/step - loss: 0.0860 - accuracy: 0.9681 - val_loss: 0.1244 - val_accuracy: 0.9729
Epoch 7/50
422/422 [==============================] - 109s 258ms/step - loss: 0.0603 - accuracy: 0.9746 - val_loss: 0.0990 -

In [ ]:
import json
with open('pos_model_architecture_early_stopping.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Load the model
model = load_model('pos_model_with_early_stopping.h5')

def predict_with_confidence(sentence):
    words = sentence.split()
    word_indices = [word2idx.get(word, word2idx["UNK"]) for word in words]
    word_indices_padded = pad_sequences([word_indices], maxlen=maxlen, padding='post')
    predictions = model.predict(word_indices_padded)

    predicted_tags = []
    confidence_scores = []

    for pred in predictions[0]:
        tag_idx = np.argmax(pred)
        predicted_tags.append(idx2tag[tag_idx])
        confidence_scores.append(pred[tag_idx])

    return predicted_tags[:len(words)], confidence_scores[:len(words)]

In [ ]:
# Example usage
sentence = "Amosi we munaanie Bibilia 23"
predicted_tags, confidence_scores = predict_with_confidence(sentence)
print("Predicted Tags:", predicted_tags)
print("Confidence Scores:", confidence_scores)

1/1 [==============================] - 1s 919ms/step
Predicted Tags: ['N', 'PREP', 'N', 'N', 'NUM']
Confidence Scores: [0.999986, 0.93083423, 0.7049353, 0.94691956, 0.9999998]


In [ ]:
with open('pos_model_architecture.json', 'w') as f:
    f.write(model.to_json())